### Задание
**Стандартная версия**
Нужно реализовать rest api на базе flask в google colab.

1. выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. Реализовать ноутбук с сервером
3. Реализовать ноутбук с клиентом


In [61]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [62]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 
           'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'] 

df = pd.read_csv("adult.data", header=None, names = columns)
df.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [64]:
binary_to_numbers = {' <=50K': 0, ' >50K': 1}

df['income'] = df['income'].replace(binary_to_numbers)
df.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0


In [65]:
df['income'].value_counts()

0    24720
1     7841
Name: income, dtype: int64

In [66]:
categorical_columns = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 
                      'race', 'sex', 'native_country']
continuous_columns = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

In [67]:
df[continuous_columns].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.581647,13.640433,17.0,28.0,37.0,48.0,90.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
education_num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital_gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital_loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
hours_per_week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0


In [68]:
for col in categorical_columns:
    print(f'Column {col}\n', df[col].unique(), '\n')

Column workclass
 [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked'] 

Column education
 [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th'] 

Column marital_status
 [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed'] 

Column occupation
 [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv'] 

Column relationship
 [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative'] 

Column race
 [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other'] 

Column sex
 [' Male' ' Fem

Разделим данные на train/test и сохраним тестовую выборку на диск

In [69]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['income']), df['income'], test_size=0.3, random_state=42)


# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [70]:
y_train.value_counts(normalize=True)

0    0.757503
1    0.242497
Name: income, dtype: float64

In [71]:
y_test.value_counts(normalize=True)

0    0.763128
1    0.236872
Name: income, dtype: float64

In [72]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column
        self.modes = None

    def fit(self, X, y=None):
        self.modes = X.mode()
        return self
        
    def transform(self, X, y=None):
        X.loc[X['workclass'] == ' ?', 'workclass'] = self.modes['workclass']
        X.loc[X['occupation'] == ' ?', 'occupation'] = self.modes['occupation']
        X.loc[X['native_country'] == ' ?', 'native_country'] = self.modes['native_country']
        return X[self.column]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [73]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                #('Preprocessing', DataPreprocessing()),
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    
    final_transformers.append((cont_col, cont_transformer))

In [74]:
feats = FeatureUnion(final_transformers)

In [75]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', GradientBoostingClassifier(random_state=42)),
])


pipeline.fit(X_train, y_train)

CPU times: total: 5.3 s
Wall time: 5.29 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('workclass',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='workclass')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='workclass'))])),
                                                ('education',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='education')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='education'))])),
                                                ('marital_status',
                                                 Pipeline(steps=[('selec

Сохраним модель (пайплайн)

In [76]:
with open("GBClassifier_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

# Step 2 - PREDICT

### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [77]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [78]:
X_test.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,27,Private,160178,Some-college,10,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,38,United-States
1,45,State-gov,50567,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
2,29,Private,185908,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,55,United-States


In [79]:
with open('GBClassifier_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [80]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('workclass',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='workclass')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='workclass'))])),
                                                ('education',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='education')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='education'))])),
                                                ('marital_status',
                                                 Pipeline(steps=[('selec

In [81]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [82]:
preds[:10]

array([0.01732859, 0.46830575, 0.62478683, 0.03849904, 0.05070261,
       0.77043823, 0.92596434, 0.01006731, 0.05746988, 0.42491338])

In [83]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.37300666248038894, F-Score=0.721, Precision=0.706, Recall=0.737


# Step 3 - FLASK

## При внедрении

In [84]:
from flask import Flask, request, jsonify
import pandas as pd

### **Создаем сервис для обработки запросов к модели**

In [85]:
# Загружаем обученные модели
with open('GBClassifier_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [86]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [87]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
        
    age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, native_country = "", "", "", "", "", "", "", "", "", "", "", "", "", ""
    request_json = request.get_json()
    
    if request_json["age"] or request_json["age"] == 0:
        age = request_json['age']
   
    if request_json["workclass"]:
        workclass = request_json['workclass']
                
    if request_json["fnlwgt"] or request_json["fnlwgt"] == 0:
        fnlwgt = request_json['fnlwgt']
        
    if request_json["education"]:
        education = request_json['education']
        
    if request_json["education_num"]:
        education_num = request_json['education_num']
        
    if request_json["marital_status"]:
        marital_status = request_json['marital_status']
        
    if request_json["occupation"]:
        occupation = request_json['occupation']
        
    if request_json["relationship"]:
        relationship = request_json['relationship']
        
    if request_json["race"]:
        race = request_json['race']
        
    if request_json["sex"]:
        sex = request_json['sex']
        
    if request_json["capital_gain"] or request_json["capital_gain"] == 0:
        capital_gain = request_json['capital_gain']
    
    if request_json["capital_loss"] or request_json["capital_loss"] == 0:
        
        capital_loss = request_json['capital_loss']
        
    if request_json["hours_per_week"] or request_json["hours_per_week"] == 0:
        hours_per_week = request_json['hours_per_week']
        
    if request_json["native_country"]:
        native_country = request_json['native_country']
      
    preds = model.predict_proba(pd.DataFrame({"age": [age], 
                                             "workclass": [workclass], 
                                             "fnlwgt": [fnlwgt], 
                                             "education": [education], 
                                             "education_num": [education_num], 
                                             "marital_status": [marital_status], 
                                             "occupation": [occupation], 
                                             "relationship": [relationship], 
                                             "race": [race], 
                                             "sex": [sex], 
                                             "capital_gain": [capital_gain], 
                                             "capital_loss": [capital_loss], 
                                             "hours_per_week": [hours_per_week], 
                                             "native_country": [native_country]}))
    data["predictions"] = preds[:, 1][0]
    
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
OK
OK

127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:41] "POST /predict HTTP/1.1" 200 -



OK
OK
OK
OK

...

OK
OK
OK
OK


127.0.0.1 - - [11/Aug/2022 22:53:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:43] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:43] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
OK
OK


127.0.0.1 - - [11/Aug/2022 22:53:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:44] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2022 22:53:44] "POST /predict HTTP/1.1" 200 -


OK
OK
OK
